In [85]:
import sys; sys.path.append('../../src/helpers')
from data_manipulation import custom_transformers as ct
import pickle
import gensim.downloader as api
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline 
import ast

In [2]:
with open('../../models/RFC_deployment_model.pkl', 'rb') as file:  
    rfc = pickle.load(file)

In [72]:
user_input_plot_summary = '''This story is about a boy named Jim. 
Jim decided he wanted to be a pirate, but not just any pirate. No Jim
 wanted to be aQWDWQND space pirate. someone who had laser guns and could take his family across the universe
 and fight aliens 238 and protect his homeland from the evil aliens who were invading
 the solar XNIAMne system that he lived in. But d18x1m ncziomz nnwiMQ the year 19272672 far in the future nothin could have prepared jim for the fights and battles
 and stars and explosions and bombs and explodings that were coming his way.
 You will have to read more to find out what happened to Jim!'''

In [73]:
user_input_plot_summary

'This story is about a boy named Jim. \nJim decided he wanted to be a pirate, but not just any pirate. No Jim\n wanted to be aQWDWQND space pirate. someone who had laser guns and could take his family across the universe\n and fight aliens 238 and protect his homeland from the evil aliens who were invading\n the solar XNIAMne system that he lived in. But d18x1m ncziomz nnwiMQ the year 19272672 far in the future nothin could have prepared jim for the fights and battles\n and stars and explosions and bombs and explodings that were coming his way.\n You will have to read more to find out what happened to Jim!'

In [74]:
new_entry_df = pd.DataFrame(data=[user_input_plot_summary], columns=['plotSum'])

In [75]:
new_entry_df

,plotSum
0,This story is about a boy named Jim. \nJim dec...


In [55]:
word2vec_model = api.load("glove-wiki-gigaword-50")

2019-06-09 13:03:47,168 gensim.models.utils_any2vec INFO     loading projection weights from /Users/nathanfranklin/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2019-06-09 13:04:13,607 gensim.models.utils_any2vec INFO     loaded (400000, 50) matrix from /Users/nathanfranklin/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz


In [76]:
with open('../../models/LDA_model.pkl', 'rb') as file:  
    lda_model = pickle.load(file)
    
with open('../../models/tf_vect.pkl', 'rb') as file:  
    tf_vect = pickle.load(file)

In [77]:
steps = [('tokenizer', ct.Tokenizer()),
                ('filterer', ct.Filter_sentence(filter_names=True)),
                ('mean_words2vec_er',ct.MeanEmbeddingVectorizer(word2vec_model)),
                ('pruner', ct.StemmingLemming(prune_type='Porter')),
                ('joiner_transformer', ct.Joiner())]
         
pipeline = Pipeline(steps)
cleaned_summary_w_w2vec = pipeline.transform(new_entry_df)

2019-06-09 13:08:29,804 data_manipulation.custom_transformers INFO     Calling the Tokenizer transformer
2019-06-09 13:08:29,811 data_manipulation.custom_transformers INFO     Calling the filter sentence transformer
2019-06-09 13:08:29,830 data_manipulation.custom_transformers INFO     Calling the StemmingLemming transformer
2019-06-09 13:08:29,834 data_manipulation.custom_transformers INFO     Calling the Joiner transformer


In [78]:
cleaned_summary_w_w2vec

,plotSum,plotSum2vec
0,stori boy name jim jim decid want pirat pirat ...,"[0.32892950944471777, -0.04371063072201714, 0...."


In [79]:
lda_transformer = ct.LDA_Vectorizer(lda_model, tf_vect)

In [82]:
prediction_DF = lda_transformer.transform(cleaned_summary_w_w2vec)

2019-06-09 13:10:27,644 data_manipulation.custom_transformers INFO     Calling the LDA transformer


In [83]:
prediction_DF

,plotSum,plotSum2vec,plotSumLDA
0,stori boy name jim jim decid want pirat pirat ...,"[0.32892950944471777, -0.04371063072201714, 0....","[[0.15974032950567266, 0.0010000000225949546, ..."


In [115]:
plotSumLDA_list = prediction_DF['plotSumLDA'].values[0][0]
plotSumLDA_DF = pd.DataFrame([plotSumLDA_list])

In [117]:
plotSumVecList = prediction_DF['plotSum2vec'].values[0]
plotSumVecs_DF = pd.DataFrame([plotSumVecList])

In [120]:
print(plotSumLDA_DF.shape)
print(plotSumVecs_DF.shape)

(1, 20)
(1, 50)


In [121]:
newColsWord2vec = []
for columnIndex in range(len(plotSumVecs_DF.columns)):
    newColsWord2vec.append('word2vec_'+str(columnIndex+1))
plotSumVecs_DF.columns = newColsWord2vec

newColsLDA = []
for columnIndex in range(len(plotSumLDA_DF.columns)):
    newColsLDA.append('LDA_'+str(columnIndex+1))
plotSumLDA_DF.columns = newColsLDA

X = pd.concat([plotSumLDA_DF,plotSumVecs_DF], axis=1)

In [122]:
X

,LDA_1,LDA_2,LDA_3,LDA_4,LDA_5,LDA_6,LDA_7,LDA_8,LDA_9,LDA_10,...,word2vec_41,word2vec_42,word2vec_43,word2vec_44,word2vec_45,word2vec_46,word2vec_47,word2vec_48,word2vec_49,word2vec_50
0,0.15974,0.001,0.189843,0.001,0.001,0.001,0.116003,0.001,0.001,0.001,...,-0.141457,0.15074,0.046191,0.11809,0.095982,0.064091,-0.150027,-0.263996,-0.146851,0.083161


In [123]:
rfc.predict(X)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [124]:
rfc.predict_proba(X)

array([[0.  , 0.06, 0.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.04, 0.  , 0.  ,
        0.  , 0.  , 0.14, 0.  , 0.  , 0.2 , 0.24, 0.  , 0.  , 0.02, 0.  ,
        0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.61, 0.45, 0.  ,
        0.01, 0.06, 0.  , 0.  , 0.  , 0.02, 0.14]])